In [1]:
import os
import geopandas as gpd
import classified_raster
import points_into_polygons
import statistics_kram as stats

from dem_utils_import import load_json

## Prep steps

In [2]:
# Load the required files
input_path_dem  = './outputs/rasters/cropped_dem.tif'
input_slope_path = './outputs/rasters/dem_slope.tif'
input_aspect_path = './outputs/rasters/dem_aspect.tif'

## Classification procedure

North - South - East - West => 
  - 0-5m 
  - 5-10m
  - 10-15m
  - 15-20m
  - 20-25m
  - 25-30m
  - 30-40m
  - 40-90m
  => For now: Only 1900-2300 & 2300-2500 & 2500-2700
    - 1900-2000
    - 2000-2100
    - 2100-2200
    - 2200-2300
    - 2300-2400
    - 2400-2500
    - 2500-2700

In [3]:
# Height list creation
list_height = []
ini_val_height = 1900

while ini_val_height <= 2700:
  list_height.append(ini_val_height)
  if ini_val_height != 2500: 
    ini_val_height = ini_val_height + 100
  elif ini_val_height == 2500:
    ini_val_height = ini_val_height + 200

In [4]:
# Slope list creation
list_slope = []
ini_val_slope = 0

while ini_val_slope <= 90:
  list_slope.append(ini_val_slope)
  if ini_val_slope == 30: 
    ini_val_slope = ini_val_slope + 10
  elif ini_val_slope == 40:
    ini_val_slope = ini_val_slope + 50
  else: 
    ini_val_slope = ini_val_slope + 5

In [5]:
# Aspect list creation
list_aspect = [315, 45, 135, 225, 315]
list_name_aspect = []

In [6]:
# Creation of a matrix containing all the raster conditions that we want to obtain:
conditions_matrix = []
list_condition = [0, 0, 0, 0, 0, 0, 1]

for asp_deg in (range(len(list_aspect)-1)):
  list_condition = [0, 0, 0, 0, 0, 0, 1]
  list_condition[4] = list_aspect[asp_deg]
  list_condition[5] = list_aspect[asp_deg + 1]
  
  for slope_deg in (range(len(list_slope)-1)):
    list_condition = [0, 0, 0, 0, list_aspect[asp_deg], list_aspect[asp_deg + 1], 1]
    list_condition[2] = list_slope[slope_deg]
    list_condition[3] = list_slope[slope_deg + 1]

    for height in (range(len(list_height)-1)):
      list_condition = [0, 0, list_slope[slope_deg], list_slope[slope_deg + 1], list_aspect[asp_deg], list_aspect[asp_deg + 1], 1]
      list_condition[0] = list_height[height]
      list_condition[1] = list_height[height + 1]

      if list_condition[4] != 315: 
        list_condition[6] = 0
      conditions_matrix.append(list_condition)

In [7]:
# Output path list creation
path = './outputs/rasters/classified_rasters/'
conditions_output_path = []

for list in range(len(conditions_matrix)):
  string = 'north'
  # Height: 
  val_height_min = str(conditions_matrix[list][0])
  val_height_max = str(conditions_matrix[list][1])

  # Slope: 
  val_slope_min = str(conditions_matrix[list][2])
  val_slope_max = str(conditions_matrix[list][3])

  # Aspect: 
  val_aspect_min = conditions_matrix[list][4]
  val_aspect_max = conditions_matrix[list][5]
  
  if (val_aspect_min == 315 and val_height_max == 45):
    string = 'north'
  elif (val_aspect_min == 45 and val_aspect_max == 135): 
    string = 'east'
  elif (val_aspect_min == 135 and val_aspect_max == 225):
    string = 'south'
  elif (val_aspect_min == 225 and val_aspect_max == 315):
    string = 'west'

  list_name = path + val_height_min + '_' + val_height_max + '_' + val_slope_min + '_' + val_slope_max + '_' + string + '.tif'
  conditions_output_path.append(list_name)

In [8]:
# Check the length of both matrixes!
if (len(conditions_matrix) == 224):
  print('Length of the conditions matrix is OK!')
if (len(conditions_output_path) == 224): 
  print('Length of the conditions name is OK!')

Length of the conditions matrix is OK!
Length of the conditions name is OK!


## Classify all the rasters

In [ ]:
# Loop over all the 224 classified rasters:
len = 224
for rast_cond in range(len):
  classified_raster.classify_raster(input_path_dem, input_slope_path, input_aspect_path, conditions_matrix[rast_cond], conditions_output_path[rast_cond])

## Polygonize all the rasters

In [19]:
# Iterate over all the classified rasters:
classified_rasters_folder_path = './outputs/rasters/classified_rasters/'
class_rast_paths = []

for path in os.listdir(classified_rasters_folder_path):
  if os.path.isfile(os.path.join(classified_rasters_folder_path, path)):
    class_rast_paths.append(os.path.join(classified_rasters_folder_path, path))

#print('Total number of files encountered: ', len(class_rast_paths))

In [20]:
# Output path list creation
output_path = './outputs/snow_depth_analysis/grau_roig/21_03_22/'
gdf_output_path = []

for list in range(len(conditions_matrix)):
  string = 'north'
  # Height: 
  val_height_min = str(conditions_matrix[list][0])
  val_height_max = str(conditions_matrix[list][1])

  # Slope: 
  val_slope_min = str(conditions_matrix[list][2])
  val_slope_max = str(conditions_matrix[list][3])

  # Aspect: 
  val_aspect_min = conditions_matrix[list][4]
  val_aspect_max = conditions_matrix[list][5]
  
  if (val_aspect_min == 315 and val_height_max == 45):
    string = 'north'
  elif (val_aspect_min == 45 and val_aspect_max == 135): 
    string = 'east'
  elif (val_aspect_min == 135 and val_aspect_max == 225):
    string = 'south'
  elif (val_aspect_min == 225 and val_aspect_max == 315):
    string = 'west'

  list_name_gdf = output_path + val_height_min + '_' + val_height_max + '_' + val_slope_min + '_' + val_slope_max + '_' + string + '.geojson'
  gdf_output_path.append(list_name_gdf)

In [23]:
# Poluygonize and check if points are into the polygons:

snow_depth_path = './inputs/snow_depth_data/grau_roig/sd_210322.geojson'
stats_list = []
empty_list = []

for raster_path in range(len(class_rast_paths)):
  print(raster_path)
  raster = gdal.Open(class_rast_paths[raster_path])

  if (raster != None):
    rast_array = raster.GetRasterBand(1).ReadAsArray()
    if (rast_array is not None):
      polygonized_rast = points_into_polygons.polygonize_raster(class_rast_paths[raster_path])
      classified_points = points_into_polygons.points_into_polys(polygonized_rast, snow_depth_path)
      classified_points = points_into_polygons.pretty_gdf(classified_points)
  
      if classified_points.empty:
        stats_list.append(empty_list)
  
      else:
        stats.stat_analysis_sd(classified_points)
        stats_list.append(classified_points)
    else: 
      stats_list.append(empty_list)

  else:
    stats_list.append(empty_list)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223


In [24]:
#SAVE the resulting list: 
import json
with open('list_statistics.json', 'w') as list: 
  json.dump(stats_list, list)

#LOAD the list: 
with open('list_statistics.json', 'r') as list:
  x = json.load(list)

TypeError: Object of type GeoDataFrame is not JSON serializable

In [22]:
import rasterio
from osgeo import gdal
x = gdal.Open('./outputs/rasters/classified_rasters/2000_2100_0_5_east.tif')
if x == None: print('True')

y = x.GetRasterBand(1).ReadAsArray()
if (y is not None):
  print(y)

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
type(y)

NoneType